In [2]:
# import SAGA code

import SAGA
from SAGA import ObjectCuts as C
from SAGA.utils import get_sdss_bands, get_sdss_colors
from SAGA.targets import TargetSelection, calc_simple_satellite_probability

print('SAGA code version', SAGA.__version__)

SAGA code version 0.2.4


In [3]:
# import other modules

from collections import defaultdict
import numpy as np
from scipy.stats import poisson, norm
from astropy.table import Table
from easyquery import Query

In [4]:
# initialize SAGA objects (Database, HostCatalog, ObjectCatalog)
# NOTE: change the path of `root_dir` to your SAGA dropbox path

saga_database = SAGA.Database(root_dir='/home/yymao/Dropbox/Academia/Collaborations/SAGA')
saga_host_catalog = SAGA.HostCatalog(saga_database)
saga_object_catalog = SAGA.ObjectCatalog(saga_database)

## check completeness for 16 paper1 hosts

In [5]:
columns = 'HOST_NSAID SATS SPEC_Z ZQUALITY gr ri r_mag'.split()
cuts = C.basic_cut & C.gri_cut & (~C.sdss_limit)

base_paper1 = saga_object_catalog.load(hosts='paper1', cuts=cuts, columns=columns)

In [6]:
data = defaultdict(list)

for t in base_paper1:
    host_id = t['HOST_NSAID'][0]
    data['SAGA_name'].append(saga_host_catalog.id_to_name(host_id))
    data['NSAID'].append(host_id)
    data['N_gri_no_spec'].append((~C.has_spec).count(t))
    
    m = (C.is_sat.count(t)+1)/(C.has_spec.count(t)+2) * (~C.has_spec).count(t)
    data['mean_miss_uniform'].append(m)
    
    data['N_miss_1s_uniform'].append(poisson.isf(norm.sf(1.0), m))
    data['N_miss_3s_uniform'].append(poisson.isf(norm.sf(3.0), m))
    
    m = calc_simple_satellite_probability((~C.has_spec).filter(t)).sum()
    data['mean_miss_model'].append(m)
    data['N_miss_1s_model'].append(poisson.isf(norm.sf(1.0), m))
    data['N_miss_3s_model'].append(poisson.isf(norm.sf(3.0), m))
    
data = Table(data)
data.show_in_notebook()

/home/yymao/Dropbox/Academia/Collaborations/SAGA-local/code/saga/SAGA/targets/assign_targeting_score.py:13: RuntimeWarning: overflow encountered in exp
  return np.where(x > model_parameters[2], np.minimum(np.exp((x-model_parameters[3])*model_parameters[4]), model_parameters[5]), 0.0)


idx,SAGA_name,NSAID,N_gri_no_spec,mean_miss_uniform,N_miss_1s_uniform,N_miss_3s_uniform,mean_miss_model,N_miss_1s_model,N_miss_3s_model
0,Gilgamesh,166313,29,0.0234817813765,0.0,1.0,0.303176,1.0,3.0
1,Odyssey,147100,26,0.133676092545,0.0,2.0,0.0646097,0.0,2.0
2,Dune,165536,47,0.0336194563662,0.0,1.0,0.282921,1.0,3.0
3,AnaK,61945,62,0.0697412823397,0.0,2.0,0.458542,1.0,3.0
4,Narnia,132339,71,0.186842105263,1.0,2.0,0.0816284,0.0,2.0
5,OBrother,149781,88,0.244105409154,1.0,3.0,0.153209,0.0,2.0
6,StarTrek,33446,126,0.185294117647,1.0,2.0,0.525959,1.0,4.0
7,Catch22,150887,159,0.936671575847,2.0,5.0,0.217914,1.0,3.0
8,ScoobyDoo,161174,405,3.80877742947,6.0,11.0,0.534678,1.0,4.0
9,MobyDick,85746,993,3.50264550265,5.0,10.0,1.18967,2.0,6.0


## Target Selection

In [7]:
saga_target_selection = TargetSelection(saga_database)

In [8]:
targets_all = saga_target_selection.compile_preliminary_targets('all', 
                                                  cuts=(C.basic_cut & ~C.has_spec & C.gri_cut), 
                                                  columns=['HOST_NSAID', 'r_mag'])

/home/yymao/Dropbox/Academia/Collaborations/SAGA-local/code/saga/SAGA/targets/assign_targeting_score.py:13: RuntimeWarning: overflow encountered in exp
  return np.where(x > model_parameters[2], np.minimum(np.exp((x-model_parameters[3])*model_parameters[4]), model_parameters[5]), 0.0)
/home/yymao/Dropbox/Academia/Collaborations/SAGA-local/code/saga/SAGA/targets/gmm.py:64: RuntimeWarning: invalid value encountered in true_divide
  p_sat /= p_notsat


In [9]:
gmm_high_p = Query('TARGETING_SCORE >= 300.0', 'TARGETING_SCORE < 400.0')

data = defaultdict(list)

for t in targets_all:
    host_id = t['HOST_NSAID'][0]
    try:
        name = saga_host_catalog.id_to_name(host_id)
    except KeyError:
        name = ''
    data['SAGA_name'].append(name)
    data['NSAID'].append(host_id)
    data['bright'].append(C.sdss_limit.count(t))
    data['GMM_high_p'].append(gmm_high_p.count(t))
    for n in (200, 250, 350, 450):
        data['mean_miss_{}'.format(n)].append(t['P_SIMPLE'][n:].sum() if len(t) > n else 0)

data = Table(data)
data.show_in_notebook(display_length=100)

idx,SAGA_name,NSAID,bright,GMM_high_p,mean_miss_200,mean_miss_250,mean_miss_350,mean_miss_450
0,Othello,145729,0,105,0.284131407738,0.266798377037,0.237104177475,0.206756234169
1,Catch22,150887,0,17,0.0,0.0,0.0,0.0
2,Alice,140594,2,229,1.01773166656,0.739386737347,0.707082211971,0.673094391823
3,,142722,3,143,0.391233384609,0.377351999283,0.350806683302,0.31782361865
4,,150578,34,145,0.368648946285,0.347841501236,0.320305973291,0.289687126875
5,,163956,4,85,0.219741046429,0.203430622816,0.164902821183,0.127585157752
6,,165153,7,177,0.480910122395,0.458345174789,0.402512013912,0.351835846901
7,,135879,2,164,0.518700242043,0.506344199181,0.469969600439,0.416244208813
8,Bandamanna,126115,21,74,0.165192708373,0.15013858676,0.122020274401,0.0800482258201
9,,144151,1,87,0.278940677643,0.261721789837,0.200519680977,0.135404646397
